In [5]:
import sys
import os
import keras.backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense, Activation
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras import callbacks
import numpy as np
from keras.callbacks import Callback
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score
def precision(y_true, y_pred):
     true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
     predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
     precision = true_positives / (predicted_positives + K.epsilon())
     return precision
 
 
def recall(y_true, y_pred):
     true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
     possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
     recall = true_positives / (possible_positives + K.epsilon())
     return recall

def f1(y_true, y_pred, beta=1):
   if beta < 0:    
       raise ValueError('The lowest choosable beta is zero (only precision).')   
   
   # If there are no true positives, fix the F score at 0 like sklearn.    
   if K.sum(K.round(K.clip(y_true, 0, 1))) == 0:   
       return 0        
   p = precision(y_true, y_pred)   
   r = recall(y_true, y_pred)    
   bb = beta ** 2    
   fbeta_score = (1 + bb) * (p * r) / (bb * p + r + K.epsilon())   
   return fbeta_score

DEV = False
argvs = sys.argv
argc = len(argvs)

if argc > 1 and (argvs[1] == "--development" or argvs[1] == "-d"):
  DEV = False

if DEV:
  epochs = 5
else:
  epochs = 15

train_data_path = 'datasets/train'
validation_data_path = 'datasets/validation'

"""
Parameters
"""
img_width, img_height = 128, 128
batch_size = 100
samples_per_epoch = 1000
validation_steps = 300
nb_filters1 = 32
nb_filters2 = 64
conv1_size = 3
conv2_size = 2
pool_size = 2
classes_num = 12
lr = 0.0004

model = Sequential()
model.add(Convolution2D(nb_filters1, conv1_size, conv1_size, border_mode ="same", input_shape=(img_width, img_height, 3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(pool_size, pool_size)))

model.add(Convolution2D(nb_filters2, conv1_size, conv1_size, border_mode ="same", input_shape=(img_width, img_height, 3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(pool_size, pool_size)))

model.add(Convolution2D(128, conv1_size, conv1_size, border_mode ="same", input_shape=(img_width, img_height, 3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(pool_size, pool_size)))

model.add(Convolution2D(128, conv1_size, conv1_size, border_mode ="same", input_shape=(img_width, img_height, 3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(pool_size, pool_size)))

model.add(Convolution2D(256, conv1_size, conv1_size, border_mode ="same", input_shape=(img_width, img_height, 3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(pool_size, pool_size)))

model.add(Convolution2D(128, conv1_size, conv1_size, border_mode ="same"))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(pool_size, pool_size), dim_ordering='th'))

model.add(Convolution2D(64, conv2_size, conv2_size, border_mode ="same"))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(pool_size, pool_size), dim_ordering='th'))

model.add(Flatten())
model.add(Dense(256))
model.add(Activation("relu"))
model.add(Dropout(0.5))
model.add(Dense(classes_num, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy',precision,recall,f1])

train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    validation_data_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

"""
Tensorboard log
"""
log_dir = './tf-log/'
tb_cb = callbacks.TensorBoard(log_dir=log_dir, histogram_freq=0)


model.fit_generator(
    train_generator,
    samples_per_epoch=samples_per_epoch,
    epochs=epochs,
    validation_data=validation_generator,
    callbacks=[tb_cb],
    validation_steps=validation_steps)

target_dir = './models/'
if not os.path.exists(target_dir):
  os.mkdir(target_dir)
model.save('./models/model5.h5')
model.save_weights('./models/weights5.h5')

c:\users\awesome\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py:70: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), input_shape=(128, 128,..., padding="same")`
c:\users\awesome\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py:74: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), input_shape=(128, 128,..., padding="same")`
c:\users\awesome\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py:78: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 3), input_shape=(128, 128,..., padding="same")`
c:\users\awesome\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py:82: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 3), input_shape=(128, 128,..., padding="same")`
c:\users\awesome\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py:86: UserWa

Found 7251 images belonging to 12 classes.
Found 984 images belonging to 12 classes.


c:\users\awesome\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py:141: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., epochs=15, validation_steps=300, callbacks=[<keras.ca..., steps_per_epoch=10, validation_data=<keras.pre...)`


Epoch 1/15
10/10 [==============================] - 146s 15s/step - loss: 2.3891 - acc: 0.1950 - precision: 0.0256 - recall: 0.0100 - f1: 0.0144 - val_loss: 2.4287 - val_acc: 0.2134 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_f1: 0.0000e+00
Epoch 2/15
10/10 [==============================] - 120s 12s/step - loss: 2.2703 - acc: 0.2450 - precision: 0.0715 - recall: 0.0110 - f1: 0.0188 - val_loss: 2.4925 - val_acc: 0.2134 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_f1: 0.0000e+00
Epoch 3/15
10/10 [==============================] - 124s 12s/step - loss: 2.2260 - acc: 0.2510 - precision: 0.1878 - recall: 0.0220 - f1: 0.0356 - val_loss: 2.4546 - val_acc: 0.2134 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_f1: 0.0000e+00
Epoch 4/15
 9/10 [==========================>...] - ETA: 0s - loss: 2.1319 - acc: 0.2589 - precision: 0.7171 - recall: 0.0378 - f1: 0.0700     

KeyboardInterrupt: 

# Shape = 128,128
## Use predict() to get output as 1-12
## Use predictVerbose() to get output as Golf-Swing-Back,Running.etc
## Input is _'full path'_ to both functions

In [2]:
import os
import numpy as np
import keras.backend as K
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from keras.models import Sequential, load_model
def precision(y_true, y_pred):
     true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
     predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
     precision = true_positives / (predicted_positives + K.epsilon())
     return precision

def recall(y_true, y_pred):
     true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
     possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
     recall = true_positives / (possible_positives + K.epsilon())
     return recall

def f1(y_true, y_pred, beta=1):
   if beta < 0:    
       raise ValueError('The lowest choosable beta is zero (only precision).')   
   
   # If there are no true positives, fix the F score at 0 like sklearn.    
   if K.sum(K.round(K.clip(y_true, 0, 1))) == 0:   
       return 0        
   p = precision(y_true, y_pred)   
   r = recall(y_true, y_pred)    
   bb = beta ** 2    
   fbeta_score = (1 + bb) * (p * r) / (bb * p + r + K.epsilon())   
   return fbeta_score

img_width, img_height = 128,128
model_path = 'models/model2.h5'
model_weights_path = 'models/weights2.h5'
model = load_model(model_path,custom_objects={"precision":precision,"f1":f1,"recall":recall})
model.load_weights(model_weights_path)

img = {
    0:'Golf-Swing-Back',
    1:'Golf-Swing-Front',
    2:'Golf-Swing-Side',
    3:'Kicking-Front',
    4:'Kicking-Side',
    5:'Lifting',
    6:'Riding-Horse',
    7:'Running',
    8:'SkateBoarding',
    9:'Swing-Bench',
    10:'Swing-SideAngle',
    11:'Walking'
}

def predict(files):
    answer = []
    for f in files:
        x = load_img(f, target_size=(img_width,img_height))
        x = img_to_array(x)
        x = np.expand_dims(x, axis=0)
        array = model.predict(x)
        result = array[0]
        answer.append(np.argmax(result) + 1)
    for i in answer:
        print(i)

def predictVerbose(file):
    answer = []
   
    x = load_img(file, target_size=(img_width,img_height))
    x = img_to_array(x)
    x = np.expand_dims(x, axis=0)
    array = model.predict(x)
    result = array[0]
    answer = img[np.argmax(result)]
    return answer
        

from subprocess import call
path = input()

rem_x = ''
def split_video(path):
    path2 = path.split('/')
    o = []
    for i in path2:
        if i != '':
            o.append(i)
    p = '/'.join(o[:-1])
    #print(p)
    rem_x = p + "/temp"
    #print("X",rem_x)
    if not os.path.exists(rem_x):
        os.makedirs(rem_x)
    dest = rem_x + '/' + path2[-1] + '-%04d.jpg'
    #print(dest)
    call(["ffmpeg", "-i", path, dest])
    return path2[-1]

w = []
for files in os.listdir(path):
    if files.endswith('avi'):
        w.append(path+'/'+files)
#print(w)
def predictFinal(path):
    import glob
    import random
    from collections import Counter
    import shutil
    try:
#         print("Removing previous files...")
    #     os.remove()
        shutil.rmtree('/'.join(path.split('/')[:-1])+'/temp', ignore_errors=True)
    except Exception as e:
        print(e)
        pass
    vid = split_video(path)
    images = glob.glob(os.path.join('predict', 'temp', '*.jpg'))
    # print(images)
    d = []

    for i in range(0,5):
        sample = random.randint(0, len(images) - 1)
        image = images[sample]
        answer = predictVerbose(image)
        d.append(answer)

    answer = Counter(d).most_common(1)[0][0]
    print(vid,"\tshows\t",answer.upper())
for i in w:
    predictFinal(i)

predict
3528-8_70514.avi 	shows	 LIFTING
5561-7_70102.avi 	shows	 WALKING
6351-6_70000.avi 	shows	 RUNNING
7616-7_70270.avi 	shows	 GOLF-SWING-BACK
RF1-11456_70034.avi 	shows	 RUNNING
RF1-13204_70095.avi 	shows	 WALKING
RF1-13206_70024.avi 	shows	 GOLF-SWING-FRONT
RF1-13588_70046.avi 	shows	 GOLF-SWING-FRONT
